In [1]:
import torch
import torchvision.models as models
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from pathlib import Path
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch import optim
from torch import nn
import torch.nn.functional as F
from torchvision import transforms
from skimage.color import rgba2rgb
from getpass import getpass
import boto3 #using creds 
import sagemaker

In [2]:
sess = sagemaker.Session()
#bucket = "arn:aws:s3:::lnd-bucket-aa"


In [3]:
!ls

data.zip  LNDnn.ipynb  Longitudinal_Nutrient_Deficiency  __MACOSX


import boto3 # этот кусок для скачки датасета без аккаунта
from botocore import UNSIGNED
from botocore.client import Config
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

with open('data.zip', 'wb') as f:
    s3.download_fileobj('intelinair-data-releases', 'longitudinal-nutrient-deficiency/Longitudinal_Nutrient_Deficiency.zip', f)

my_key_id = getpass(prompt='your account key: ', stream=None) #нужно для подгрузки модели на S3
my_secret_key = getpass(prompt='your account secret key: ', stream=None)
s3 = boto3.client('s3',
    aws_access_key_id=my_key_id,
    aws_secret_access_key=secret_key
)
with open('Longitudinal_Nutrient_Deficiency', 'wb') as f:
    s3.download_fileobj(, 'longitudinal-nutrient-deficiency/Longitudinal_Nutrient_Deficiency.zip', f)
with open('filename', 'wb') as data:
    s3.download_fileobj('mybucket', 'mykey', data)

In [21]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

with open('data.zip', 'wb') as f:
    s3.download_fileobj('intelinair-data-releases', 'longitudinal-nutrient-deficiency/Longitudinal_Nutrient_Deficiency.zip', f)

In [22]:
!unzip -qq ./data.zip

import pandas as pd
bucket = "lnd-bucket-aa"

folder = 'Longitudinal_Nutrient_Deficiency/'

s3uri = 's3://{}/{}'.format(bucket, folder)

#df = pd.read_csv(s3uri)

#df.head()

import boto3

 When running on SageMaker, need execution role
from sagemaker import get_execution_role
role = get_execution_role()

#Declare bucket name, remote file, and destination
my_bucket = 'lnd-bucket-aa'
orig_file = 'Longitudinal_Nutrient_Deficiency/'
#dest_file = 'local/path/to/file.p'

 Connect to S3 bucket and download file
s3 = boto3.resource('s3')
s3.Bucket(my_bucket).download_file(orig_file, "./")

In [ ]:
!unzip -qq ./data.zip

In [20]:
#%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [5]:
#from torch.utils.tensorboard import SummaryWriter # TensorBoard support

#tb = SummaryWriter(comment='myboard')

# Device configuration
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda')

device.type

'cuda'

## Готовим датасет

Скачиваем данные, передаем трансформер (предобработку), визуализируем  примеры семплов, создаем dataloaders для обучения и тестирования модели

In [24]:
from torch.utils.data import Dataset

import cv2
import PIL
class CustomDataset(Dataset):
    def __init__(self, img_dir, transform=None):
     #   self.img_labels = os.walk(self.img_dir)][1:]
        self.img_labels = [x[0]+"/nutrient_mask_g0.png" for x in os.walk(datasetDir)][1:]
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
     #   lst = [x[0] for x in os.walk(datasetDir)][1:][]
      #  folder_list = [x[0] for x in os.walk(self.img_dir)][1:]
#pil_image = PIL.Image.open('Image.jpg').convert('RGB') 

        image = PIL.Image.open(os.path.join(datasetDir, "field_"+"{:03d}".format(idx+1)+"/image_i0.png")).convert('RGB') #убираем ф
        label =  PIL.Image.open(os.path.join(datasetDir, "field_"+"{:03d}".format(idx+1)+"/nutrient_mask_g0.png")).convert('1')
   #     image = rgba2rgb(PIL.Image.open(os.path.join(datasetDir, "field_"+"{:03d}".format(idx+1)+"/image_i0.png")))
    #    label =  rgba2rgb(PIL.Image.open(os.path.join(datasetDir, "field_"+"{:03d}".format(idx+1)+"/nutrient_mask_g0.png")))


        if self.transform:
            image = self.transform(image)#[:,:,:3]
            label = self.transform(label)#[:,:,:3]

        return image, label

In [25]:
import os
datasetDir = os.path.join(os.getcwd(),'Longitudinal_Nutrient_Deficiency')
mydataset = CustomDataset(img_dir = datasetDir, transform = transforms.Compose([transforms.Resize(256)
    , transforms.CenterCrop(256),transforms.ToTensor()]))

In [26]:
from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(mydataset, test_size=0.2, random_state=25)


In [27]:
train_loader =  torch.utils.data.DataLoader(training_data, 
                                          batch_size=120, 
                                          shuffle=True, 
                                          num_workers=1)
    
test_loader = torch.utils.data.DataLoader(testing_data, 
                                          batch_size=120, 
                                          shuffle=True, 
                                          num_workers=1)


In [28]:
import time
def train(model, train_dl, valid_dl, loss_fn, optimizer, acc_fn, epochs=20):
    start = time.time()
    model = model.to(device)

    train_loss, valid_loss = [], []

    best_acc = 0.0

    for epoch in range(epochs):
        print('Epoch {}/{}'.format(epoch, epochs - 1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train(True)  # Set trainind mode = true
                dataloader = train_dl
            else:
                model.train(False)  # Set model to evaluate mode
                dataloader = valid_dl
            running_loss = 0.0
            running_acc = 0.0
            step = 0            
            for x, y in dataloader:

                x = x.to(device)
                y = y.to(device)
                step += 1
                if phase == 'train':
                    optimizer.zero_grad()
                    outputs = model(x)
                    loss = loss_fn(outputs, y)
                    loss.backward()
                    optimizer.step()
                else:
                    with torch.no_grad():
                        outputs = model(x)
                        loss = loss_fn(outputs, y.long())
                acc = acc_fn(outputs, y)
                running_acc  += acc*dataloader.batch_size
                running_loss += loss*dataloader.batch_size 
            epoch_loss = running_loss / len(dataloader.dataset)
            epoch_acc = running_acc / len(dataloader.dataset)

            print('{} Loss: {:.4f} Acc: {}'.format(phase, epoch_loss, epoch_acc))

            train_loss.append(epoch_loss) if phase=='train' else valid_loss.append(epoch_loss)

    time_elapsed = time.time() - start
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    
    
    return train_loss, valid_loss    

def acc_metric(predb, yb):
    return (predb.argmax(dim=1) == yb.cpu()).float().mean()

In [29]:
import numpy
def numpy_iou(outs,labels, threshold=0.5):# IoU calculation
    outs = (outs > threshold)
    intersection = numpy.logical_and(outs.cpu().detach().numpy(), labels.cpu().detach().numpy())
    union = numpy.logical_or(outs.cpu().detach().numpy(), labels.cpu().detach().numpy())
    iou_score = numpy.sum(intersection) / numpy.sum(union)
    return iou_score

In [13]:
!pip install segmentation_models_pytorch

     |████████████████████████████████| 88 kB 5.3 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 58 kB 1.5 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 376 kB 28.8 MB/s            
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-py3-none-any.whl size=12419 sha256=96c51f4763d859d5035539a6176824bf4f477eddd131043ef4af224a756079a0
  Stored in directory: /home/ec2-user/.cache/pip/wheels/1c/07/d1/ff77968842daae1dde944173a8e8a7be193646d37842f13b24
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60963 sha256=deef6bf2bcbb429df40eda3fb2a90249a69fe597c26874ca6af267ee4bde1e35
  Stored in directory: /home/ec2-user/.cache/pip/wheels/9c/03/81/494596f1d017bbf80daf53bc33dbb05aaa5a6ad6d44c74011f
Successfully built efficientnet-pytorch pretrainedmodels


In [30]:
import segmentation_models_pytorch as smp
loss_fn = smp.losses.TverskyLoss("binary", alpha=2, beta=5, gamma=5)

model = smp.DeepLabV3Plus(
    encoder_name="timm-efficientnet-b1",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=3,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)

In [31]:
device = torch.device('cpu')

device

device(type='cpu')

In [32]:
opt = torch.optim.AdamW(model.parameters(),lr=0.001)

train_loss, valid_loss = train(model,train_loader,test_loader,loss_fn,opt,numpy_iou,epochs=2)
#torch.save(model.state_dict(), "tversky_loss_SMOL_5_epochs_iou_10fields_DeepLab_NOTquantized.h5")

Epoch 0/1
----------


OSError: [Errno 12] Cannot allocate memory

In [17]:
unet = UNET(3,1)
loss_fn = DiceLoss()
opt = torch.optim.Adam(unet.parameters(),lr=0.01)
train_loss, valid_loss = train(unet,train_loader,test_loader,loss_fn,opt,iou_pytorch,epochs=20)
torch.save(unet.state_dict(), "Dice_20epochs_iou.h5")
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, '')).upload_file('Dice_20epochs_iou.h5')

Epoch 0/0
----------


/Users/jetpacula/miniforge3/envs/firstEnv/lib/python3.9/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


train Loss: 0.9269 Acc: 0.06267140060663223
valid Loss: 0.9036 Acc: 0.06853586435317993
Training complete in 10m 32s


NameError: name 'model' is not defined

In [47]:
tb.close()
%tensorboard --logdir=runs

loss_fn = FocalLoss
opt = torch.optim.Adam(unet.parameters(),lr=0.01)
train_loss, valid_loss = train(unet,train_loader,test_loader,loss_fn,opt,iou_pytorch,epochs=5)

